# Getting Links from URLs in People Records

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('..')

from heritageconnector.utils import data_loaders
from heritageconnector.entity_matching import lookup

import os

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook().pandas();

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_FOLDER = "../GITIGNORE_DATA"

## 1. Import Data

In [3]:
loader = data_loaders.local_loader()
people_df = loader.load_mimsy_people_data()

In [4]:
people_df.head(2)

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE
0,10243,Brooklyn Arms Company,NaN,NaN,Brooklyn Arms Company,NaN,NaN,N,Y,NaN,object record: 1987-1020,c. 1870,"Brooklyn, New York, New York state, United States",NaN,NaN,NaN,American,manufacturer of mathematical instruments,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,06-NOV-18
1,10245,"Zenthon, Edward Rupert",NaN,Edward Rupert,Zenthon,NaN,NaN,M,Y,REF: http://www.iwm.org.uk/collections/item/object/1030031461,NaN,1920-07,"London, Greater London, England, United Kingdom",c. 2002,NaN,NaN,British,engineer,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,05-AUG-15


## 2. Test URL to WID Method

In [5]:
df = people_df.copy()

In [7]:
# create joined column
cols_look_for_url = ["DESCRIPTION", "NOTE"]
df[cols_look_for_url] = df[cols_look_for_url].astype(str)
df["res_ALL_NOTES"] = df[cols_look_for_url].agg(' --- '.join, axis=1)

# match URLs
url_match = "((?:https?:\/\/|www\.|https?:\/\/|www\.)[a-z0-9\.:].*?(?=[\s;]|$))"

lk = lookup.wikidata_id()

def text_to_urls(text):
    return lk.get_from_free_text(text, return_urls=True)

df["res_WIKIDATA_IDs"], df["res_URLS"] = zip(*df["res_ALL_NOTES"].progress_apply(text_to_urls))

KeyboardInterrupt: 